<a href="https://colab.research.google.com/github/Sumaiya379/AI-and-ML/blob/main/CNN_ASsignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import numpy as np
import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 1. Load pretrained ResNet18 model
def load_pretrained_model(num_classes=2):
    """Load pretrained ResNet18 and replace final layer"""
    model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

    # 2. Replace the final fully connected layer
    num_features = model.fc.in_features
    model.fc = nn.Linear(num_features, num_classes)

    return model.to(device)

## 3. Freeze all layers except the final one
def freeze_model_layers(model):
    for param in model.parameters():
        param.requires_grad = False
    # Unfreeze the final layer
    for param in model.fc.parameters():
        param.requires_grad = True

# Data transformations with augmentation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load dataset (assuming you have train/val/test folders)
data_dir = 'data'  # Replace with your dataset path
try:
    # Create datasets
    image_datasets = {
        'train': ImageFolder(os.path.join(data_dir, 'train'), data_transforms['train']),
        'val': ImageFolder(os.path.join(data_dir, 'val'), data_transforms['val']),
        'test': ImageFolder(os.path.join(data_dir, 'test'), data_transforms['test'])
    }

    # Create dataloaders
    dataloaders = {
        'train': DataLoader(image_datasets['train'], batch_size=32, shuffle=True, num_workers=4),
        'val': DataLoader(image_datasets['val'], batch_size=32, shuffle=False, num_workers=4),
        'test': DataLoader(image_datasets['test'], batch_size=32, shuffle=False, num_workers=4)
    }

    dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
    class_names = image_datasets['train'].classes

    print(f"Dataset sizes - Train: {dataset_sizes['train']}, Val: {dataset_sizes['val']}, Test: {dataset_sizes['test']}")
    print(f"Class names: {class_names}")

except FileNotFoundError:
    print("Error: Dataset folder structure not found. Please ensure you have 'train', 'val', and 'test' folders in your data directory.")
    exit()

# Initialize model
model = load_pretrained_model(num_classes=2)
freeze_model_layers(model)

# Loss function and optimizer (only trains unfrozen parameters)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

# Training loop with validation
def train_model(model, criterion, optimizer, scheduler, num_epochs=15):
    best_acc = 0.0
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            history[f'{phase}_loss'].append(epoch_loss)
            history[f'{phase}_acc'].append(epoch_acc.item())

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(model.state_dict(), 'best_model_resnet18.pth')
                print("New best model saved!")

    print(f'Best val Acc: {best_acc:.4f}')
    return history

# Train the model
print("Starting training...")
history = train_model(model, criterion, optimizer, scheduler, num_epochs=15)

# Plot training results
def plot_results(history):
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='Train Loss')
    plt.plot(history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='Train Accuracy')
    plt.plot(history['val_acc'], label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.savefig('training_curves.png')
    plt.show()

plot_results(history)

## 4. Evaluate on test data
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (preds == labels).sum().item()

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

    # Classification report
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig('confusion_matrix.png')
    plt.show()

    return accuracy

# Load best model and evaluate on test set
print("\nEvaluating on test set...")
model.load_state_dict(torch.load('best_model_resnet18.pth'))
test_accuracy = evaluate_model(model, dataloaders['test'])

# Save the complete model (architecture + weights)
torch.save(model, 'final_resnet18_model.pth')
print(f"Final model saved with test accuracy: {test_accuracy:.2f}%")

Using device: cuda
Error: Dataset folder structure not found. Please ensure you have 'train', 'val', and 'test' folders in your data directory.


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 68.2MB/s]


Starting training...
Epoch 1/15
----------


NameError: name 'dataloaders' is not defined

In [ ]:
# ============================
# 1. Install & Import Packages
# ============================
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

# ============================
# 2. Data Preparation (CIFAR-10)
# ============================
batch_size = 64
img_size = 224  # Resize for VGG/ResNet

transform_train = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

transform_test = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class_names = train_dataset.classes
print("Classes:", class_names)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ============================
# 3. Training & Evaluation Function
# ============================
def train_and_evaluate(model, epochs=2):
    # Freeze feature extractor
    for param in model.parameters():
        param.requires_grad = False

    # Replace classifier
    if hasattr(model, 'fc'):  # ResNet
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, len(class_names))
    else:  # VGG
        num_features = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_features, len(class_names))

    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        print(f"Epoch {epoch+1}/{epochs}, "
              f"Loss: {running_loss/len(train_loader):.4f}, "
              f"Train Acc: {100*correct/total:.2f}%")

    # Evaluation
    model.eval()
    correct, total = 0, 0
    all_images, all_preds, all_labels = [], [], []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            # Save for plotting
            all_images.append(inputs.cpu())
            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())

    acc = 100 * correct / total
    print(f"Test Accuracy: {acc:.2f}%")
    return model, acc, torch.cat(all_images), torch.cat(all_preds), torch.cat(all_labels)

# ============================
# 4. Run Transfer Learning for ResNet18
# ============================
print("\n--- Training ResNet18 ---")
resnet_model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
resnet_model, resnet_acc, resnet_imgs, resnet_preds, resnet_labels = train_and_evaluate(resnet_model, epochs= 2)

# ============================
# 6. Function to Show Images (Actual vs Predicted)
# ============================
def imshow(img, title=None):
    img = img.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean  # Unnormalize
    img = np.clip(img, 0, 1)
    plt.imshow(img)
    if title:
        plt.title(title)
    plt.axis('off')

def show_predictions(images, labels, preds, n=10):
    plt.figure(figsize=(15, 8))
    for i in range(n):
        ax = plt.subplot(2, n//2, i+1)
        imshow(images[i], title=f"Actual: {class_names[labels[i]]}\nPred: {class_names[preds[i]]}")
    plt.tight_layout()
    plt.show()

# ============================
# 7. Plot Predictions for ResNet18
# ============================
print("\nResNet18 - Actual vs Predicted")
show_predictions(resnet_imgs, resnet_labels, resnet_preds, n=10)

# ============================
# 8. Plot Predictions for VGG16
# ============================
print("\nVGG16 - Actual vs Predicted")
show_predictions(vgg_imgs, vgg_labels, vgg_preds, n=10)

# ============================
# 9. Compare Results
# ============================
print(f"\nResNet18 Test Accuracy: {resnet_acc:.2f}%")
print(f"VGG16 Test Accuracy: {vgg_acc:.2f}%")


100%|██████████| 170M/170M [00:12<00:00, 13.2MB/s]


Classes: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

--- Training ResNet18 ---
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 198MB/s]


Epoch 1/2, Loss: 0.8265, Train Acc: 73.33%
Epoch 2/2, Loss: 0.6179, Train Acc: 78.90%
Test Accuracy: 79.12%


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models

# Load CIFAR-10 dataset and filter for 2 classes (e.g., cats and dogs)
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load training data (filter classes 3: cat, 5: dog)
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_indices = [i for i, (_, label) in enumerate(train_set) if label in [3, 5]]
train_subset = torch.utils.data.Subset(train_set, train_indices)
train_loader = torch.utils.data.DataLoader(train_subset, batch_size=32, shuffle=True)

# Load test data similarly
test_set = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_indices = [i for i, (_, label) in enumerate(test_set) if label in [3, 5]]
test_subset = torch.utils.data.Subset(test_set, test_indices)
test_loader = torch.utils.data.DataLoader(test_subset, batch_size=32, shuffle=False)

# Load pretrained ResNet18
model = models.resnet18(pretrained=True)

# Replace final layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

# Freeze all layers except the classifier
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        # Adjust labels: cat(3) -> 0, dog(5) -> 1
        labels = torch.where(labels == 3, torch.tensor(0), torch.tensor(1)).to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader):.4f}")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        labels = torch.where(labels == 3, torch.tensor(0), torch.tensor(1)).to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")